# import module

In [1]:
from sklearn import tree
import pandas as pd
import numpy as np
import os as oss
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_validate, KFold, train_test_split
from sklearn.metrics import accuracy_score, cohen_kappa_score, make_scorer, f1_score, recall_score
from sklearn.preprocessing import MinMaxScaler, minmax_scale

import pydotplus, pprint
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.linear_model import LogisticRegression

from catboost import CatBoostClassifier, CatBoost
import lightgbm as lgb
import xgboost as xgb
from tqdm import tqdm
import glob
import pickle

/home/nishiolab/.local/lib/python3.6/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


# main

# データいじいじ

In [6]:
step_count=pd.read_csv('../../etc/step_num_UT_cont/features.csv')
gradient=pd.read_csv('../../etc/gradient_UT/features.csv')
datas=pd.read_csv('../../etc/windows_features/clipped_std_features.csv')

datas['SC_M']=step_count['SC_M']
datas['SCF_M']=step_count['SCF_M']
datas['SCP_M']=step_count['SCP_M']
datas['gradient_M']=gradient['gradient_M']
datas['SC_H']=step_count['SC_H']
datas['SCF_H']=step_count['SCF_H']
datas['SCP_H']=step_count['SCP_H']
datas['gradient_H']=gradient['gradient_H']

datas=datas[datas['user']!="7NM9zimotoB"]
datas=datas[datas['user']!="7NM2zimotoD"]
datas=datas[datas['user']!="6NM9yossi"]

In [7]:
import warnings
warnings.simplefilter('ignore')

moving=datas[datas['label']=='m']
observing=datas[datas['label']=='o']
searching=datas[datas['label']=='s']

moving['index']=0
searching['index']=1
observing['index']=2

moving['indexMO']=0
searching['indexMO']=0
observing['indexMO']=1

In [4]:
moving_train, moving_test=train_test_split(moving, train_size=6000, random_state=0)
observing_train, observing_test=train_test_split(observing, train_size=6000, random_state=0)
searching_train, searching_test=train_test_split(searching, train_size=6000, random_state=0)

df_train=pd.concat([moving_train, searching_train, observing_train])
df_test=pd.concat([moving_test, searching_test, observing_test])

In [152]:
moving_train=moving[moving['user']!='5NM4tiger']
observing_train=observing[observing['user']!='5NM4tiger']
searching_train=searching[searching['user']!='5NM4tiger']

moving_test=moving[moving['user']=='5NM4tiger']
observing_test=observing[observing['user']=='5NM4tiger']
searching_test=searching[searching['user']=='5NM4tiger']

df_train=pd.concat([moving_train, searching_train, observing_train])
df_test=pd.concat([moving_test, searching_test, observing_test])

In [153]:
x_train=df_train.drop(['label', 'user', 'index', 'indexMO'], axis=1)
y_train=df_train[['index', 'indexMO']]
x_test=df_test.drop(['label', 'user', 'index', 'indexMO'], axis=1)
y_test=df_test[['index', 'indexMO']]

In [154]:
x_train=x_train[['acc_std_window0', 'acc_x_std_window0', 'gyro_y_std_window0',
       'gyro_x_std_window1', 'acc_y_std_window2', 'acc_z_std_window2',
       'acc_kurtosis_window3', 'acc_skewness_window3', 'acc_std_window3',
       'acc_z_std_window3', 'SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]

x_test=x_test[['acc_std_window0', 'acc_x_std_window0', 'gyro_y_std_window0',
       'gyro_x_std_window1', 'acc_y_std_window2', 'acc_z_std_window2',
       'acc_kurtosis_window3', 'acc_skewness_window3', 'acc_std_window3',
       'acc_z_std_window3', 'SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]

# 分類

In [161]:
from sklearn import svm, metrics, preprocessing

clf = svm.SVC(kernel='rbf', gamma=0.6, C=0.6, class_weight='balanced')
clf.fit(x_train, y_train[['indexMO']])

clf_pr=clf.predict(x_test)
ac_score=metrics.accuracy_score(y_test[['indexMO']], clf_pr)

print(ac_score)
#print(confusion_matrix(y_test, clf_pr))

y_test['pred_index']=clf_pr

x2_test=x_test[y_test['pred_index']==1]
y2_test=y_test[y_test['pred_index']==1][['index']]

clf2 = svm.SVC(kernel='rbf', gamma=0.6, C=0.6, class_weight='balanced')
clf2.fit(x_train, y_train[['index']])

clf2_pr=clf2.predict(x2_test)
ac_score=metrics.accuracy_score(y2_test, clf2_pr)

y2_test['pred2_index']=clf2_pr

y2_test['index']==y2_test['pred2_index']

0.9037656903765691


767     True
768     True
769     True
770     True
771     True
       ...  
919    False
920    False
964    False
965    False
966    False
Length: 184, dtype: bool

In [166]:
o_index=pd.DataFrame({'index':y_test[y_test['pred_index']==2]['pred_index']})

In [164]:
ms_index=pd.DataFrame({'index':y2_test['pred2_index']})

In [168]:
o_index.loc[o_index['index']==2, 'index']=3

In [173]:
p=pd.concat([o_index, ms_index])

In [176]:
o_index

,index
765,3
766,3
879,3
880,3
881,3
956,3
957,3
958,3
959,3
1009,3


In [194]:
ms_index.loc(0)[767]

index    1
Name: 767, dtype: int64

In [209]:
o_index.loc(0)[766]

index    3
Name: 766, dtype: int64

In [210]:
for k in o_index['Name']:
    print(k)

KeyError: 'Name'

# result

In [162]:
result=pd.DataFrame()

result['result']=y2_test['index']==y2_test['pred2_index']

a=pd.DataFrame(y_test[y_test['pred_index']==2]['indexMO']==y_test[y_test['pred_index']==2]['pred_index'], columns=['result'])

result=pd.concat([result, a])

len(result[result['result']==True])/len(result)

0.7238493723849372

# k-fold

In [9]:
from sklearn.model_selection import KFold
from sklearn import svm, metrics, preprocessing

DATA=pd.concat([moving, searching, observing])

In [54]:
kf=KFold(n_splits=10, shuffle=True, random_state=1)
scores_m=[]
scores_s=[]
scores_o=[]

for train_index, test_index in kf.split(DATA):
    df_train = DATA.iloc[train_index]
    df_test = DATA.iloc[test_index]
    
    x_train=df_train.drop(['label', 'user', 'index', 'indexMO'], axis=1)
    y_train=df_train[['index', 'indexMO']]
    x_test=df_test.drop(['label', 'user', 'index', 'indexMO'], axis=1)
    y_test=df_test[['index', 'indexMO']]
    
    #x_train=x_train[['SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]
    #x_test=x_test[['SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]
    x_train=x_train[['acc_std_window0', 'acc_x_std_window0', 'gyro_y_std_window0',
       'gyro_x_std_window1', 'acc_y_std_window2', 'acc_z_std_window2',
       'acc_kurtosis_window3', 'acc_skewness_window3', 'acc_std_window3',
       'acc_z_std_window3', 'SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]

    x_test=x_test[['acc_std_window0', 'acc_x_std_window0', 'gyro_y_std_window0',
       'gyro_x_std_window1', 'acc_y_std_window2', 'acc_z_std_window2',
       'acc_kurtosis_window3', 'acc_skewness_window3', 'acc_std_window3',
       'acc_z_std_window3', 'SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]
    
    clf = svm.SVC(kernel='rbf', gamma=0.6, C=0.6, class_weight='balanced')
    clf.fit(x_train, y_train[['indexMO']])
    clf_pr=clf.predict(x_test)

    y_test['pred_index']=clf_pr

    x2_test=x_test[y_test['pred_index']==0]
    y2_test=y_test[y_test['pred_index']==0][['index']]
    
    clf2 = svm.SVC(kernel='rbf', gamma=0.6, C=0.6, class_weight='balanced')
    x_train = x_train[(y_train['index']==0) | (y_train['index']==1)]
    #x_train = x_train[['SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]
    y_train = y_train[(y_train['index']==0) | (y_train['index']==1)]
    clf2.fit(x_train, y_train[['index']])
    #clf2_pr=clf2.predict(x2_test[['SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']])
    clf2_pr=clf2.predict(x2_test)

    y2_test['pred2_index']=clf2_pr
    
    #result['result'] = y2_test['index']==y2_test['pred2_index']
    #a=pd.DataFrame(y_test[y_test['pred_index']==1]['indexMO']==y_test[y_test['pred_index']==1]['pred_index'], columns=['result'])
    #results=pd.concat([result, a])
    
    #Search精度確認用
    result_m=pd.DataFrame()
    y2_test_m=y2_test[y2_test['index']==0]
    result_m['result'] = y2_test_m['index']==y2_test_m['pred2_index']
    scores_m+=[len(result_m[result_m['result']==True])/len(df_test[df_test['index']==0])]
    
    result_s=pd.DataFrame()
    y2_test_s=y2_test[y2_test['index']==1]
    result_s['result'] = y2_test_s['index']==y2_test_s['pred2_index']
    scores_s+=[len(result_s[result_s['result']==True])/len(df_test[df_test['index']==1])]
    
    result_o=pd.DataFrame()
    y_test_o=y_test[y_test['index']==2]
    result_o['result'] = y_test_o['indexMO']==y_test_o['pred_index']
    scores_o+=[len(result_o[result_o['result']==True])/len(df_test[df_test['index']==2])]

In [35]:
sum(scores)/len(scores)

0.7261153071475621

In [55]:
print(sum(scores_m)/len(scores_m))
print(sum(scores_s)/len(scores_s))
print(sum(scores_o)/len(scores_o))

0.7458170275893299
0.5531092132463259
0.895087517292756


In [42]:
print(len(moving), len(searching), len(observing))

11000 6755 8088


In [46]:
np.array(scores).std()

0.012155043834011307